In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

In [3]:
batch_size=64
epochs=100
latent_dim=256 #Size within which the data is represented after being encoded
num_samples=10000
data_path='/content/fra.txt'

# Data Preparation

In [4]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

In [5]:
with open(data_path,"r",encoding='utf-8') as f:
  lines=f.read().split("\n")

In [6]:
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text,_=line.split("\t")#it is expected  each line in the file contains an input and target sequence separated by a tab character,'_' is used to discard any additional parts after the target sequence

  target_text="\t"+target_text+"\n"#common technique in sequence to sequence learning to inform the decoder when to start and stop generating the target text
  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_characters:#to build the vocabulary for the input and target text data. 
      input_characters.add(char)

  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)    

In [7]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters) #to determine the size of input and output layers
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [8]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])
#Doctionaries which map each character to a integer index,to represent data in numerical format for getting processed by the model

In [9]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")

#initializing arrays with zeroes. These arrays will hold the data and will be used to train the model

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0 #filling the left over space with blanks to have same length for all sequences
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0#padding
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Model Building

In [11]:
encoder_inputs=keras.Input(shape=(None,num_encoder_tokens)) #input length of the  variable sequence is not known beforehand, and the other argument is for specifying the size of the one hot encoded vectors
encoder=keras.layers.LSTM(latent_dim,return_state=True)#instructing the lstm layer to return the final hidden state and the final state
encoder_outputs,state_h,state_c=encoder(encoder_inputs)

In [12]:
encoder_states=[state_h,state_c]

#discard the encoder output but keept the states(context vector)

In [13]:
decoder_inputs=keras.Input(shape=(None,num_decoder_tokens))

In [14]:
decoder_lstm=keras.layers.LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=keras.layers.Dense(num_decoder_tokens,activation="softmax")
#This dense layer transforms the high dimensional output from the decoder outputs to something usable i.e., creates a probability distribution over the possible target tokens
decoder_outputs=decoder_dense(decoder_outputs)

model=keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)

# Training

In [15]:
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 70)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 93)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        334848      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [16]:
model.fit(
    [encoder_input_data,decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

model.save("s2s")

Epoch 1/100
125/125 [==============================] - 11s 26ms/step - loss: 1.2275 - accuracy: 0.7326 - val_loss: 1.0695 - val_accuracy: 0.7138
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.9418 - accuracy: 0.7475 - val_loss: 0.9880 - val_accuracy: 0.7139
Epoch 3/100
125/125 [==============================] - 1s 10ms/step - loss: 0.8510 - accuracy: 0.7641 - val_loss: 0.8811 - val_accuracy: 0.7445
Epoch 4/100
125/125 [==============================] - 1s 11ms/step - loss: 0.7530 - accuracy: 0.7890 - val_loss: 0.8270 - val_accuracy: 0.7633
Epoch 5/100
125/125 [==============================] - 2s 12ms/step - loss: 0.6736 - accuracy: 0.8066 - val_loss: 0.7265 - val_accuracy: 0.7888
Epoch 6/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6303 - accuracy: 0.8168 - val_loss: 0.6850 - val_accuracy: 0.7992
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6024 - accuracy: 0.8233 - val_loss: 0.6562 - val_accuracy

# Sampling

In [17]:
model=keras.models.load_model("s2s")

In [18]:
encoder_inputs=model.input[0]
encoder_outputs,state_h_enc,state_c_enc=model.layers[2].output
encoder_states=[state_h_enc,state_c_enc]
encoder_model=keras.Model(encoder_inputs,encoder_states)

In [19]:
decoder_inputs=model.input[1]
decoder_state_input_h=keras.Input(shape=(latent_dim,))
decoder_state_input_c=keras.Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_lstm=model.layers[3]
decoder_outputs,state_h_dec,state_c_dec=decoder_lstm(
    decoder_inputs,initial_state=decoder_states_inputs
)#decoding the english using the context vectors from the encoding layer

In [20]:
decoder_states=[state_h_dec,state_c_dec]#Dense Layer
decoder_dense=model.layers[4]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=keras.Model(
    [decoder_inputs]+ decoder_states_inputs, [decoder_outputs]+decoder_states
)#(input,output)

In [21]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
#Mapping the indices back to the respective characters

In [22]:
def decode_sequence(input_seq):
  states_value=encoder_model.predict(input_seq)

  target_seq=np.zeros((1,1,num_decoder_tokens))


  stop_condition=False
  decoded_sentence=""

  while not stop_condition:
    output_tokens,h,c=decoder_model.predict([target_seq]+states_value)

    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    sampled_char=reverse_target_char_index[sampled_token_index]
    decoded_sentence+=sampled_char

    if sampled_char=="\n" or len(decoded_sentence)>max_decoder_seq_length:
      stop_condition=True

    
    target_seq=np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index]=1.0

    states_value=[h,c]

  return decoded_sentence




In [25]:
for seq_index in range(20):

  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print("-")

  print("Input Sentence: ",input_texts[seq_index])
  print("decoded sentence: ",decoded_sentence)

1/1 [==============================] - 0s 23ms/step
-
Input Sentence:  Go.
decoded sentence:  Cassez-vous.

1/1 [==============================] - 0s 28ms/step
-
Input Sentence:  Go.
decoded sentence:  Cassez-vous.

1/1 [==============================] - 0s 18ms/step
-
Input Sentence:  Go.
decoded sentence:  Cassez-vous.

1/1 [==============================] - 0s 20ms/step
-
Input Sentence:  Go.
decoded sentence:  Cassez-vous.

1/1 [==============================] - 0s 18ms/step
-
Input Sentence:  Hi.
decoded sentence:  Salut.

1/1 [==============================] - 0s 19ms/step
-
Input Sentence:  Hi.
decoded sentence:  Salut.

1/1 [==============================] - 0s 18ms/step
-
Input Sentence:  Run!
decoded sentence:  Cours !

1/1 [==============================] - 0s 18ms/step
-
Input Sentence:  Run!
decoded sentence:  Cours !

1/1 [==============================] - 0s 22ms/step
-
Input Sentence:  Run!
decoded sentence:  Cours !

1/1 [==============================] - 0s 20ms/step
